In [1]:
'''
!pip3 install dash
!pip3 install jupyter_dash
!pip3 install numpy
!pip3 install pandas
!pip3 install dash_bootstrap_components
!pip3 install sklearn
!pip3 install osmnx
!pip3 install utm
!pip3 uninstall umap
!pip3 install umap-learn -Y
'''

'''
conda config --prepend channels conda-forge
conda create -n ox --strict-channel-priority osmnx jupyter
conda activate ox
conda install -c conda-forge dash
conda install -c plotly jupyter-dash
conda install -c conda-forge dash-bootstrap-components
conda install -c conda-forge utm
conda install -c conda-forge umap-learn
'''

'\nconda config --prepend channels conda-forge\nconda create -n ox --strict-channel-priority osmnx jupyter\nconda activate ox\nconda install -c conda-forge dash\nconda install -c plotly jupyter-dash\nconda install -c conda-forge dash-bootstrap-components\nconda install -c conda-forge utm\nconda install -c conda-forge umap-learn\n'

In [2]:
import os
import CityHub
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import math
from statistics import mean
from sklearn.manifold import TSNE

In [3]:
# carrega o citygraph preprocessado
ch=CityHub.load_preprocessed_CityHub('data/xsp-sibgrapi2022_allSP.bin')

In [4]:
#carrega o array x de features estáticas e a tabela de indexação de esquinas dos arquivos csv

#X_static = pd.read_csv("data/embeddings_12features.csv", header=None)
X_static = pd.read_csv("data/embeddings_29012024135109_I.csv", header=None)
tabela_esquinas_bbox = list(X_static.index)
X_static

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,5.736410e+05,-0.626002,-1.268893,1.206740,-1.778689,0.225697,-0.166080,0.126586,-0.099920,0.335917,...,-0.388108,1.621543,2.998554,0.228687,-0.741924,-1.292939,-1.037105,-0.322883,-0.166774,0.437212
1,5.736430e+05,-0.283219,-0.331197,0.704825,-0.339171,0.247755,0.070938,0.322802,0.081962,-0.312682,...,0.666763,-0.600094,0.145135,-0.050602,0.211130,0.453759,0.144385,0.057386,0.080566,-0.134860
2,5.736440e+05,0.272857,0.233490,0.027325,-0.057056,0.199632,-0.154925,-0.109882,-0.217054,-0.485391,...,0.440411,-0.314662,0.159484,-0.354617,0.423847,-0.214864,-0.395497,-0.220674,0.289730,-0.158690
3,5.772390e+05,-0.034954,-0.050377,0.185070,0.221263,0.152630,0.026533,0.095878,0.002866,-0.388435,...,0.341524,-0.079986,0.225151,-0.029506,0.367493,-0.027915,0.088139,-0.083764,0.201071,-0.189486
4,5.821390e+05,0.165602,0.361338,-0.230509,0.297033,0.240721,-0.329641,0.165279,-0.280456,-0.131055,...,-0.065690,0.160051,0.401181,0.046327,0.171209,-0.052240,1.054048,0.181105,-0.072819,-0.409285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119743,8.851304e+09,0.022974,-0.177165,0.017482,-0.201867,0.216619,-0.220000,0.043191,-0.095257,0.154332,...,0.018874,-0.279222,-0.331450,-0.063619,-0.144458,-0.211404,-0.275220,0.125350,0.229133,0.099090
119744,8.851395e+09,0.044985,-0.076225,0.136130,-0.219721,0.001890,0.024031,-0.152839,-0.204604,0.341380,...,0.033314,-0.221956,0.006123,0.283367,0.197640,-0.120110,-0.141406,0.374881,0.051472,-0.026254
119745,8.851395e+09,-0.114926,0.050571,0.064060,0.025185,-0.132474,-0.118022,-0.039051,-0.127691,0.156768,...,0.230146,-0.540163,0.072802,0.177605,0.029144,-0.209792,-0.133482,0.109555,-0.070855,0.013165
119746,8.866803e+09,0.020527,-0.283842,-0.088737,-0.583167,-0.325995,-0.256150,-0.200202,-0.008290,-0.171682,...,-0.249999,0.106281,-0.071770,0.245917,-0.416999,0.182555,0.214470,0.166665,0.277125,0.262515


In [5]:
# gerando df para com features estáticas e dinâmicas
# é possível incluir initial_date e final_date em compute_dynamic_feature_df
# df = pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,'2020-01-01','2020-12-31')], axis = 1)
df = X_static.copy()
static_feature_names = X_static.columns[0:51] #13
dynamic_feature_names = []

In [6]:
# Redução de dimensionalidade UMAP, a primeira com todas as features
import umap.umap_ as umap
reducer = umap.UMAP()
df_normalized=(df-df.mean())/df.std()
df_normalized=df_normalized.fillna(0)
X_embedded = reducer.fit_transform(df_normalized)

C:\Users\XIMENA\anaconda3\envs\cityhub\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\XIMENA\anaconda3\envs\cityhub\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\XIMENA\anaconda3\envs\cityhub\lib\site-packages\umap\distances.py:1086: Num

In [7]:
df['lat'] =[ch.city_vert_list[i][0] for i in tabela_esquinas_bbox]
df['lon'] = [ch.city_vert_list[i][1] for i in tabela_esquinas_bbox]
df['u'] = X_embedded[:,0]
df['v'] = X_embedded[:,1]

In [8]:
# Calcule histogramas e distâncias entre o subconjunto selecionado e o total
def compute_histograms(df_name,latlong = True, nbins = 18):
    hists = dict()
    bins = dict()
    if latlong:
        columns = df_name.columns[:-4]
    else:
        columns = df_name.columns[:]
    for col in columns:
        hists[col],bins[col]=np.histogram(df_name[col], bins=nbins, density=True)
    return hists,bins

In [9]:
full_hists,full_bins = compute_histograms(df)

In [10]:
def compute_histograms_definedbins(df_name, definedbins,latlong = True):
    hists = dict()
    bins = dict()
    if latlong:
        columns = df_name.columns[:-4]
    else:
        columns = df_name.columns[:]
    for col in columns:
        hists[col],bins[col]=np.histogram(df_name[col], bins=definedbins[col], density=True)
    return hists,bins

In [11]:
partial_hists,partial_bins = compute_histograms_definedbins(df,full_bins)

In [12]:
# calculate the kl divergence
#def kl_divergence(p, q):
#   return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))
from scipy.stats import wasserstein_distance

def kl(p, q):
    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0.0))


def hist_euc(p, q):
    return np.linalg.norm(p/np.linalg.norm(p)-q/np.linalg.norm(q))


def dict_divergence(dict1,dict2):
    dict_div = dict()
    for k, v in dict1.items():
        dict_div[k]=hist_euc(v,dict2[k])    
    #   dict_div[k]=wasserstein_distance(v,dict2[k])
    #    if k=='Crimes':
    #    print(dict_div[k])
    return dict_div

In [13]:
df_backup=df
df_backup

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,lat,lon,u,v
0,5.736410e+05,-0.626002,-1.268893,1.206740,-1.778689,0.225697,-0.166080,0.126586,-0.099920,0.335917,...,-0.741924,-1.292939,-1.037105,-0.322883,-0.166774,0.437212,-23.562991,-46.703965,0.868245,0.188894
1,5.736430e+05,-0.283219,-0.331197,0.704825,-0.339171,0.247755,0.070938,0.322802,0.081962,-0.312682,...,0.211130,0.453759,0.144385,0.057386,0.080566,-0.134860,-23.567465,-46.702229,-1.037238,-3.868628
2,5.736440e+05,0.272857,0.233490,0.027325,-0.057056,0.199632,-0.154925,-0.109882,-0.217054,-0.485391,...,0.423847,-0.214864,-0.395497,-0.220674,0.289730,-0.158690,-23.572962,-46.699235,-1.114328,-4.634959
3,5.772390e+05,-0.034954,-0.050377,0.185070,0.221263,0.152630,0.026533,0.095878,0.002866,-0.388435,...,0.367493,-0.027915,0.088139,-0.083764,0.201071,-0.189486,-23.571831,-46.702075,-1.019307,-3.901389
4,5.821390e+05,0.165602,0.361338,-0.230509,0.297033,0.240721,-0.329641,0.165279,-0.280456,-0.131055,...,0.171209,-0.052240,1.054048,0.181105,-0.072819,-0.409285,-23.534616,-46.748148,-2.593682,0.015863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119743,8.851304e+09,0.022974,-0.177165,0.017482,-0.201867,0.216619,-0.220000,0.043191,-0.095257,0.154332,...,-0.144458,-0.211404,-0.275220,0.125350,0.229133,0.099090,-23.649217,-46.667740,5.080869,0.590535
119744,8.851395e+09,0.044985,-0.076225,0.136130,-0.219721,0.001890,0.024031,-0.152839,-0.204604,0.341380,...,0.197640,-0.120110,-0.141406,0.374881,0.051472,-0.026254,-23.651471,-46.662832,5.643357,0.568182
119745,8.851395e+09,-0.114926,0.050571,0.064060,0.025185,-0.132474,-0.118022,-0.039051,-0.127691,0.156768,...,0.029144,-0.209792,-0.133482,0.109555,-0.070855,0.013165,-23.651152,-46.663291,5.097335,0.460714
119746,8.866803e+09,0.020527,-0.283842,-0.088737,-0.583167,-0.325995,-0.256150,-0.200202,-0.008290,-0.171682,...,-0.416999,0.182555,0.214470,0.166665,0.277125,0.262515,-23.500164,-46.611595,3.051640,-4.446597


In [14]:
input_features = pd.read_csv("data/nodos_subgrafoSPdaily_DEMOG.csv")
tabela_esquinas_bbox = list(input_features.index)
input_features = input_features.iloc[:, 0:13]

input_features

,Nodo,Renda_media_por_domicilio,Renda_media_responsaveis,Responsaveis_sem_renda_taxa,Alfabetizados_de_7_a_15_anos,menores_de_18_anos_taxa,18_a_65_anos_taxa,maiores_de_65_anos_taxa,Pontos_de_onibus,Estacao_de_metro,Estacao_de_trem,Terminal_de_onibus,Favela_proxima
0,573641,17703.084229,10920.004934,0.066325,0.991071,0.210939,0.631893,0.157168,0.0,1.0,3.0,1.0,0.0
1,573643,3331.532558,2068.113256,0.119709,0.906177,0.169365,0.648065,0.182571,0.0,3.0,3.0,1.0,0.0
2,573644,12941.995652,9115.711180,0.077019,1.000000,0.125065,0.742215,0.132719,0.0,3.0,3.0,1.0,0.0
3,577239,12695.832272,9088.416954,0.100086,1.000000,0.117566,0.794885,0.087549,0.0,2.0,4.0,1.0,0.0
4,582139,3720.941215,1987.940523,0.141944,1.000000,0.190903,0.668093,0.141004,0.0,0.0,4.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119743,8851304020,7041.454106,4075.824972,0.167410,0.986301,0.187652,0.714181,0.098166,0.0,0.0,0.0,1.0,0.0
119744,8851395327,3553.659402,1986.580743,0.100226,0.989130,0.179016,0.698658,0.122326,0.0,0.0,0.0,1.0,1.0
119745,8851395328,3126.708842,1629.332049,0.112995,0.968254,0.206792,0.680242,0.112966,0.0,0.0,0.0,1.0,1.0
119746,8866803226,3242.424042,1772.676811,0.115751,0.962429,0.183829,0.656725,0.159446,0.0,0.0,0.0,1.0,0.0


# Main code

In [15]:
import dash
from dash import dcc, ctx
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pddict_divergence
import plotly.express as px
import plotly.graph_objs as go
import dash_bootstrap_components as dbc
from datetime import date

html_dash = """
<!DOCTYPE html>
<html>
   <head>
       {%metas%}
       <title>{%title%}</title>
       {%favicon%}
       {%css%}
       <style>

            .DateInput_input__focused{
                border-bottom: 2px solid #0175ff;
            }
            .DateInput_input, .DateInput_input_1 {
                font-family: Arial;
                font-size: 10px; 
                height: 10px;
            }
            .DateInput, .DateInput_1 {
                width: 40%;
                height: inherit;
            }
       </style>
   </head>
   <body>
       {%app_entry%}
       <footer>
           {%config%}
           {%scripts%}
           {%renderer%}
       </footer>
   </body>
</html>
"""

In [16]:
#app = JupyterDash()
selected_corners_indices = []
app = JupyterDash(external_stylesheets=[dbc.themes.PULSE])
server = app.server
app.index_string = html_dash
df=df_backup.copy()
# Configuração de janela padrão
fig = px.scatter_mapbox(df, lat="lat", lon="lon", hover_name = df.index, hover_data=df.columns[:-2],
                        color_discrete_sequence=['#00a699'], zoom=3, height=300)#"fuchsia"
fig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450
                 )

scatter = go.Figure()
scatter.add_trace(go.Scattergl(
                x = df.u,
                y = df.v,
                mode = 'markers',
                marker = dict(
                color='#00a699', #green 
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
))
scatter.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    dragmode='select',
    clickmode='event+select')

dict_div=dict_divergence(full_hists,full_hists)
data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
fig_barplot = go.Figure(data=data)
fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
fig_barplot.update_xaxes(tickfont_size=10)

def blank_fig(text):
    fig = go.Figure(go.Scatter(x=[], y = []))
    fig.update_layout(
          title_text=text,title_font_size=10,
        template = None,height=30)
    fig.update_xaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_yaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_layout({"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"})
    return fig

# Estrutura das janelas
app.layout = dbc.Container(
    [
        html.H1(),
        html.Center(html.H4(children='CityHub Visual Explorer')),
        
        html.Hr(),
         dbc.Row(
                [
                    dbc.Col([
                        dbc.Row(html.H5(children='Features')),
                        dbc.Row(html.H6(children='Static')), 
                        dbc.Row(          
                           dcc.Checklist(
                               id="features_checklist_static",
                               options=static_feature_names,
                               value=static_feature_names,
                               style={'font-size': '10px'},
                               labelStyle={'display': 'block',} ),                
                        ),
                        dbc.Row(html.H6(children='Dynamic')), 
                        dbc.Row(
                            dcc.DatePickerRange(
                                id='my-date-picker-range',
                                start_date=date(2020, 1, 1),
                                end_date=date(2020, 12, 31),
                                min_date_allowed = date(2020, 1, 1),
                                max_date_allowed = date(2020, 12, 31),
                            )
                        ),
                        html.H2(),
                        dbc.Row(          
                           dcc.Checklist(
                               id="features_checklist_dynamic",
                               options=dynamic_feature_names,
                               value=dynamic_feature_names,
                               style={'font-size': '10px'},
                               labelStyle={'display': 'block',} ),                
                        ),
                        html.H2(),
                        dbc.Button(
                            str('Update'),
                            className="btn btn-primary",
                            id='update_button',
                            n_clicks=0,
                        )
                        ], width=2),
                    dbc.Col([
                        dbc.Row(html.H5(children='Projection View')),                     
                        dbc.Row(dcc.Graph(id="projectionScatter", figure=scatter)),
                            ], width=5),
                    dbc.Col([
                        dbc.Row(html.H5(children='Map View')),
                        dbc.Row(dcc.Graph(id="mymap", figure=fig))
                        ], width=5),
                ]
            ),
        dbc.Row(
                [
                    dbc.Col([
                        dbc.Row(html.H5(children='Features distribution')),  
                        dbc.Row(html.H6(children='Global')), 
                        dbc.Row(dcc.Graph(id="general",figure= blank_fig(''))),
                        dbc.Row(html.H6(children='Selected')),  
                        dbc.Row(dcc.Graph(id="detail",figure= blank_fig(''))),
                            ], width=2),
                    dbc.Col([
                        dbc.Row(html.H5(children='Features distribution census')),
                        dbc.Row(dcc.Graph(id="barplot_census", figure=blank_fig(''))
                        ),
                    ], width=10),
                ]
            ),
        
        dcc.Store(id="features-detail-df"),
        dcc.Store(id="features-general-df"),
        dcc.Store(id="input_features")
    ]
)


C:\Users\XIMENA\anaconda3\envs\cityhub\lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [17]:
# Projection View
def scatterplot(x, y, title="", axis_type="Linear"):
    return {
        "data": [go.Scatter(x=df.u, y=df.v, mode="markers") ],
    }

# @app.callback atualiza as visualizações, tem entrada, saída e função
@app.callback(
    Output("barplot_census","figure"),
    Output("projectionScatter", "figure"),
    Output("mymap", "figure"),
    Output('features-detail-df', 'data'),
    Output('features-general-df', 'data'),
    [
        Input("features_checklist_static", "value"),
        Input("features_checklist_dynamic", "value"),
        Input('my-date-picker-range', 'start_date'),
        Input('my-date-picker-range', 'end_date'),
        Input("projectionScatter", "selectedData"),
        Input("projectionScatter", "figure"),
        Input("barplot_census", "figure"),
        Input("mymap", "selectedData"),
        Input("mymap", "figure"),
        Input('features-detail-df', 'data'),
        Input('features-general-df', 'data'),
        Input('update_button', 'n_clicks'),
    ], prevent_initial_call=True
)
def update_cb(options_chosen_static, options_chosen_dynamic, start_date, end_date, selectedData, 
              scatter_current_fig, barplot_census_current_fig, selectedDataMap, 
              map_current_fig,features_current_data,features_current_data_general,n_clicks):
    triggered_id = ctx.triggered_id
    global selected_corners_indices
    if triggered_id == 'mymap':
        if len(selectedDataMap["points"])==0:
            return barplot_census_current_fig,scatter_current_fig,map_current_fig,features_current_data,features_current_data_general
        # Se você tiver features dinâmicos, eles deverão ser calculados e concatenados
        #df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        df2=pd.concat([X_static,df.lat,df.lon,df.u,df.v], axis = 1)
        temp=df2
        pt = selectedDataMap["points"]
        #indices = []
        selected_corners_indices.clear()
        for sel in pt:
            #indices.append(sel['hovertext'])
            selected_corners_indices.append(sel['hovertext'])
        selected_corners_indices = list(set(selected_corners_indices))
        #temp=temp.loc[indices]
        temp=temp.iloc[selected_corners_indices]
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(temp,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        """fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)"""
        
        #CENSUS NEW
        temp_census=input_features.iloc[selected_corners_indices]
        full_hists_census,full_bins_census = compute_histograms(input_features, False)
        partial_hists_census,partial_bins_census = compute_histograms_definedbins(temp_census,full_bins_census, False)
        dict_div_census=dict_divergence(full_hists_census,partial_hists_census)
        print('keys', dict_div_census.keys())
        data_census = [go.Bar(x=list(dict_div_census.keys()),y=list(dict_div_census.values()))]
        fig_barplot_census = go.Figure(data=data_census)
        fig_barplot_census.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot_census.update_xaxes(tickfont_size=10)
            
        
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = df.u,#scatter_current_fig['data'][0]['x'],
                y = df.v,#scatter_current_fig['data'][0]['y'],
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend=False) 
        return fig_barplot_census,scatter,map_current_fig,temp.to_json(date_format='iso', orient='split'), df2.to_json(date_format='iso', orient='split')

    elif triggered_id == 'projectionScatter':
        print('projectionScatter')
        #df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        df2=pd.concat([X_static,df.lat,df.lon,df.u,df.v], axis = 1)
        #options_chosen = options_chosen_static + options_chosen_dynamic
        temp = df2
        temp["lat"] = df["lat"]
        temp["lon"] = df["lon"]
        pt = pd.DataFrame(selectedData["points"])
        pt = pt[pt.curveNumber == 0]
        #indices = []
        selected_corners_indices.clear()
        for sel in pt['pointIndex']:
            #indices.append(sel['pointIndex'])
            selected_corners_indices.append(sel)
        #temp=temp.iloc[indices]
        selected_corners_indices = list(set(selected_corners_indices))
        temp=temp.iloc[selected_corners_indices]
        mapfig = px.scatter_mapbox(temp, lat="lat", lon="lon", hover_name=temp.index, hover_data=temp.columns[:-2],
                        color_discrete_sequence=['#636efa'], zoom=3, height=300)#'blue'
        mapfig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450
                 )
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = df2.u,#scatter_current_fig['data'][0]['x'],
                y = df2.v,#scatter_current_fig['data'][0]['y'],
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend=False) 
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(temp,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        """fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)"""
        
        #CENSUS NEW
        temp_census=input_features.iloc[selected_corners_indices]
        full_hists_census,full_bins_census = compute_histograms(input_features,False)
        partial_hists_census,partial_bins_census = compute_histograms_definedbins(temp_census,full_bins_census,False)
        dict_div_census=dict_divergence(full_hists_census,partial_hists_census)
        print('keys', dict_div_census.keys())
        data_census = [go.Bar(x=list(dict_div_census.keys()),y=list(dict_div_census.values()))]
        fig_barplot_census = go.Figure(data=data_census)
        fig_barplot_census.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot_census.update_xaxes(tickfont_size=10)
        
        
        return fig_barplot_census,scatter,mapfig,temp.to_json(date_format='iso', orient='split'),df2.to_json(date_format='iso', orient='split')
    
    elif triggered_id == 'update_button': #or triggered_id =='features_checklist_static' or triggered_id == 'features_checklist_dynamic' or triggered_id == 'my-date-picker-range':
        #df2=pd.concat([X_static, compute_dynamic_feature_df(tabela_esquinas_bbox,start_date,end_date),df.lat,df.lon,df.u,df.v], axis = 1)
        df2=pd.concat([X_static,df.lat,df.lon,df.u,df.v], axis = 1)
        options_chosen = options_chosen_static + options_chosen_dynamic
        temp = df2.filter(options_chosen)
        temp_normalized=(temp-temp.mean())/temp.std()
        temp_normalized.fillna(0)
        X_embedded = reducer.fit_transform(temp_normalized)
        temp['u'] = X_embedded[:,0]
        temp['v'] = X_embedded[:,1]
        df.u=temp.u
        df.v=temp.v
        df2.u=temp.u
        df2.v=temp.v
        selected_df = df2.iloc[selected_corners_indices]
        scatter = go.Figure()
        scatter.add_trace(go.Scattergl(
                x = temp.u,
                y = temp.v,
                mode = 'markers',
                marker = dict(
                color='#00a699',#'green',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.add_trace(go.Scattergl(
                x = selected_df.u,
                y = selected_df.v,
                mode = 'markers',
                marker = dict(
                #color='#5a3685',#'blue',
                color = 'red',
                size=3,
                opacity=0.5
                )#,
                #selected_marker_color='red'
        ))
        scatter.update_layout(clickmode='event+select',
                              dragmode='select',
                              margin ={'l':0,'t':0,'b':0,'r':0},
                              showlegend = False)       
        mapfig = px.scatter_mapbox(selected_df, lat="lat", lon="lon", hover_name=selected_df.index,  hover_data=selected_df.columns[:-2],
                        color_discrete_sequence=['#636efa'], zoom=3, height=300)#"blue"
        mapfig.update_layout(mapbox_style="open-street-map",
                  margin={"r":0,"t":0,"l":0,"b":0},
                  autosize=True,
                  mapbox = {
                'center': {'lon': -46.625290, 'lat': -23.533773},
                'zoom': 10},
                  height=450,
                 )  
        full_hists,full_bins = compute_histograms(df2)
        partial_hists,partial_bins = compute_histograms_definedbins(selected_df,full_bins)
        dict_div=dict_divergence(full_hists,partial_hists)
        data = [go.Bar(x=list(dict_div.keys()),y=list(dict_div.values()))]
        """fig_barplot = go.Figure(data=data)
        fig_barplot.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot.update_xaxes(tickfont_size=10)"""
        
        #CENSUS NEW
        temp_census=input_features.iloc[selected_corners_indices]
        full_hists_census,full_bins_census = compute_histograms(input_features,False)
        partial_hists_census,partial_bins_census = compute_histograms_definedbins(temp_census,full_bins_census,False)
        dict_div_census=dict_divergence(full_hists_census,partial_hists_census)
        print('keys', dict_div_census.keys())
        data_census = [go.Bar(x=list(dict_div_census.keys()),y=list(dict_div_census.values()))]
        fig_barplot_census = go.Figure(data=data_census)
        fig_barplot_census.update_layout(margin ={'l':0,'t':0,'b':0,'r':0},
                  height=400)
        fig_barplot_census.update_xaxes(tickfont_size=10)
        
        
        return fig_barplot_census,scatter,mapfig,selected_df.to_json(date_format='iso', orient='split'),df2.to_json(date_format='iso', orient='split')
    else:
        return barplot_census_current_fig,scatter_current_fig,map_current_fig,features_current_data,features_current_data_general
    
# Atualiza Features distribution View (Global y Selected) ao clicar nas barras
@app.callback(
    Output('detail', 'figure'),
    Output('general', 'figure'),
    # entradas que la función espera para ejecutarse. 
    Input('barplot_census', 'clickData'),
    Input('features-detail-df', 'data'),
    Input('features-general-df', 'data'),
    Input('input_features', 'data'), #census data
    Input('detail', 'figure'),
    Input('general', 'figure'),
    
    prevent_initial_call=True  # evita que la función se ejecute cuando la aplicación se carga por primera vez
)
def display_click_data_2(clickData, df_selected_json, df_global_json, df_census, detail_current_fig, general_current_fig):
    
    print('Imprimiendo string', type(input_features))
    
    if clickData is None:
        print('clickData None')
        return detail_current_fig, general_current_fig
   
    census_df = input_features
    name = clickData['points'][0]['label']
    print('name', name)

    try:
        range_min = census_df[name].min()
        range_max = census_df[name].max()
    except:
        range_min = 100000
        range_max = -1
        
    print('ranges', range_max , '-' , range_min)
    print('pruebaaa YOYOK', census_df[name])
    
    geral_xx = census_df[name]
    
    step = (range_max - range_min)/10
    
    geral_fig = go.Figure([go.Histogram(x=geral_xx,xbins=dict(size=step))])
    geral_fig.update_xaxes(range=[range_min-step, range_max+step])
    geral_fig.update_layout(bargap=0.3)
    print('step', step)
    geral_fig.update_layout(title_text=name,title_font_size=10,
                        margin ={'l':0,'t':20,'b':0,'r':0},height=150)
    
    # Selected / detail
    
    selected_df = pd.read_json(df_selected_json, orient='split') 
    print('arrive 0', selected_df)
    
    temp_census=input_features.iloc[selected_corners_indices]
    print(' ===== necesito saber si esta var existe aqui', temp_census)  # checar Notes 
    
    detail_xx = temp_census[name].tolist()
    detail_yy = temp_census[temp_census.columns[1]].tolist()


    print('detail_yy',detail_yy)
    print('detail_xx',detail_xx)  # probar
    print('geral_xx', geral_xx)

    yyy = []
    num = 0
    
    
    for i in geral_xx:
        if i in detail_xx:
            if (len(detail_yy) <= num):
                break
            yyy.append(detail_yy[num])
            num +=1
        
        else:
            print('else')
            yyy.append(0)
        print('loop ',str(i))
    
    #print('++ ++', yyy)
    common_x = np.array(geral_xx).astype(str)
    print(common_x)

    detail_current_fig = go.Figure([go.Bar(x=common_x, y=yyy)])
    detail_current_fig.update_xaxes(range=[range_min-step, range_max+step])
    detail_current_fig.update_layout(bargap=0.3)
    detail_current_fig.update_layout(title_text=name,title_font_size=10,
                      margin ={'l':0,'t':20,'b':0,'r':0},height=150)

    #geral_fig = go.Figure([go.Bar(x=common_x, y=geral_yy)])
    return detail_current_fig, geral_fig


# Run server
if __name__ == "__main__":
    # app.run_server(debug=True,mode="inline")
    app.run_server()
    #df=df_backup
"""   
if __name__ == '__main__':
    app.run_server(debug=True, port=805) # or whatever you choose
    """ 

Dash app running on http://127.0.0.1:8050/


"   \nif __name__ == '__main__':\n    app.run_server(debug=True, port=805) # or whatever you choose\n    "

In [18]:
from IPython.display import Audio
import numpy as np

# Definir la frecuencia y la duración del sonido
frecuencia = 150  # Hz
frecuencia2 = 100
duracion = 2000  # milisegundos

# Crear un array de tiempo
tiempo = np.linspace(0, duracion / 10000, duracion)

# Crear la señal de audio (una onda sinusoidal)
sonido = np.sin(2 * np.pi * frecuencia * tiempo)

sonido2 = np.sin( 2 * np.pi * frecuencia2 * tiempo)

sonido = np.concatenate((sonido, sonido2))

# Reproducir el sonido al finalizar la ejecución
Audio(sonido, rate=44100, autoplay=True)

In [19]:
print('hola')

hola


name = 'Renda_media_por_domicilio'
temp_census=input_features.iloc[selected_corners_indices]
detail_yy = temp_census[temp_census.columns[1]].tolist()
detail_xx = temp_census[name].tolist()
geral_xx = input_features[name]
geral_xx

detail_xx

yyy = []
num = 0
        
for i in geral_xx:
    if i in detail_xx: 
        print('ok', num, 'i = ', i)
        if (len(detail_yy) <= num):
            break
        yyy.append(detail_yy[num])
        print(detail_yy[num])
        num +=1
        
    else:
        yyy.append(0)
    #print('loop ',str(i), 'num', num)
    

yyy[0]